In [1]:
import os, pickle
import equinox as eqx
import xcquinox as xce

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


# The functions below should be considered as development tools. They have been implemented into the base xcquinox package, but can be modified here to fit your purposes or have their functionality expanded.

In [62]:
def make_net(xorc, level, depth, nhidden, ninput = None, use = None, spin_scaling = None, lob = None, ueg_limit = None,
                random_seed = None, savepath = None, configfile = 'network.config'):
    '''
    make_net is a utility function designed to easily create new, individual exchange or correlation networks with ease. If no extra arguments are specified, the network will be generated with a default structure that respects the various constraints implemented within xcquinox

    :param xorc: 'X' or 'C' -- the type of network to generate, exchange or correlation
    :type xorc: str
    :param level: one of ['GGA', 'MGGA', 'NONLOCAL', 'NL'], indicating the desired rung of Jacob's Ladder. NONLOCAL = NL
    :type level: str
    :param depth: The number of hidden layers in the generated network.
    :type depth: int
    :param nhidden: The number of nodes in a hidden layer
    :type nhidden: int
    :param ninput: The number of inputs the network will expect, defaults to None for automatic selection based on level
    :type ninput: int, optional
    :param use: The indices of the descriptors to evaluate the network on, defaults to None
    :type use: list of ints, optional
    :param spin_scaling: Whether or not to enforce the spin-scaling contraint in the generated network, defaults to None
    :type spin_scaling: bool, optional
    :param lob: Lieb-Oxford bound: If non-zero (i.e., truthy), the output values of e_x or e_c will be squashed between [-1, lob-1], defaults to None
    :type lob: float, optional
    :param ueg_limit: Whether or not to enforce the UEG scaling constraint, defaults to None
    :type ueg_limit: bool, optional
    :param random_seed: The random seed to use in generating initial network weights, defaults to None
    :type random_seed: int, optional
    :param savepath: Location to save the generated network and associated config file, defaults to None
    :type savepath: str, optional
    :param configfile: Name for the configuration file, needed when reading in the network to re-generate the same structure, defaults to 'network.config'
    :type configfile: str, optional
    :return: The resulting exchange or correlation network.
    :rtype: :xcquinox.net.eX: or :xcquinox.net.eC:
    '''
    defaults_dct = {'GGA': {'X': {'ninput' : 1, 'depth': 3, 'nhidden': 16, 'use': [1], 'spin_scaling': True, 'lob': 1.804, 'ueg_limit':True},
                            'C': {'ninput': 1, 'depth': 3, 'nhidden': 16, 'use': [2], 'spin_scaling': False, 'lob': 2.0, 'ueg_limit':True}
                           },
                    'MGGA': {'X': {'ninput' : 2, 'depth': 3, 'nhidden': 16, 'use': [1, 2], 'spin_scaling': True, 'lob': 1.174, 'ueg_limit':True},
                            'C': {'ninput': 2, 'depth': 3, 'nhidden': 16, 'use': [2, 3], 'spin_scaling': False, 'lob': 2.0, 'ueg_limit':True}
                           },
                    'NL': {'X': {'ninput' : 15, 'depth': 3, 'nhidden': 16, 'use': None, 'spin_scaling': True, 'lob': 1.174, 'ueg_limit':True},
                            'C': {'ninput': 16, 'depth': 3, 'nhidden': 16, 'use': None, 'spin_scaling': False, 'lob': 2.0, 'ueg_limit':True}
                           }
                   }
    assert level.upper() in ['GGA', 'MGGA', 'NONLOCAL', 'NL']

    ninput = ninput if ninput is not None else defaults_dct[level.upper()][xorc.upper()]['ninput']
    depth = depth if depth is not None else defaults_dct[level.upper()][xorc.upper()]['depth']
    nhidden = nhidden if nhidden is not None else defaults_dct[level.upper()][xorc.upper()]['nhidden']
    use = use if use is not None else defaults_dct[level.upper()][xorc.upper()]['use']
    spin_scaling = spin_scaling if spin_scaling is not None else defaults_dct[level.upper()][xorc.upper()]['spin_scaling']
    ueg_limit = ueg_limit if ueg_limit is not None else defaults_dct[level.upper()][xorc.upper()]['ueg_limit']
    lob = lob if lob is not None else defaults_dct[level.upper()][xorc.upper()]['lob']
    random_seed = random_seed if random_seed is not None else 92017
    config = {'ninput':ninput,
              'depth':depth,
              'nhidden':nhidden,
              'use':use,
              'spin_scaling':spin_scaling,
              'ueg_limit': ueg_limit,
              'lob':lob,
             'random_seed': random_seed}
    if xorc.upper() == 'X':    
        net = xce.net.eX(n_input=ninput, use=use, depth=depth, n_hidden=nhidden, spin_scaling=spin_scaling, lob=lob, seed=random_seed)
    elif xorc.upper() == 'C':
        net = xce.net.eC(n_input=ninput, use=use, depth=depth, n_hidden=nhidden, spin_scaling=spin_scaling, lob=lob, seed=random_seed)
    
    if savepath:
        try:
            os.makedirs(savepath)
        except Exception as e:
            print(e)
            print(f'Exception raised in creating {savepath}.')
        with open(os.path.join(savepath, configfile), 'w') as f:
            for k, v in config.items():
                f.write(f'{k}\t{v}\n')
        with open(os.path.join(savepath, configfile+'.pkl'), 'wb') as f:
            pickle.dump(config, f)
        eqx.tree_serialise_leaves(os.path.join(savepath, 'xc.eqx'), net)

    return net, config

def get_net(xorc, level, net_path, configfile='network.config', netfile='xc.eqx'):
    '''
    A utility function to easily load in a previously generated network. Functionally creates a random network of the same architecture, then overwrites the weights with those of the saved network.

    :param xorc: 'X' or 'C' -- the type of network to generate, exchange or correlation
    :type xorc: str
    :param level: one of ['GGA', 'MGGA', 'NONLOCAL', 'NL'], indicating the desired rung of Jacob's Ladder. NONLOCAL = NL
    :type level: str
    :param net_path: Location of the saved network. Must have a {configfile}.pkl parameter file within.
    :type net_path: str
    :param configfile: Name for the configuration file, needed when reading in the network to re-generate the same structure, defaults to 'network.config'
    :type configfile: str, optional
    :param netfile: Name for the network file, needed when reading in the network overwrite generated random weights, defaults to 'xc.eqx'. If Falsy, just generates random network based on config file.
    :type netfile: str, optional
    :return: The requested exchange or correlation network.
    :rtype: :xcquinox.net.eX: or :xcquinox.net.eC:
    '''
    with open(os.path.join(net_path, configfile+'.pkl'), 'rb') as f:
        params = pickle.load(f)
    #network parameters
    depth = params['depth']
    nodes = params['nhidden']
    use = params['use']
    inp = params['ninput']
    ss = params['spin_scaling']
    lob = params['lob']
    ueg = params['ueg_limit']
    seed = params['random_seed']

    net, _ = make_net(xorc=xorc, level=level, depth=depth, nhidden=nodes, ninput=inp, use=use,
                       spin_scaling = ss, lob = lob, ueg_limit = ueg, random_seed = seed, configfile=configfile)
    if netfile:
        net = eqx.tree_deserialise_leaves(os.path.join(net_path, netfile), net)
    return net


def make_xcfunc(level, x_net_path, c_net_path, configfile = 'network.config', 
                xdsfile = 'xc.eqx', cdsfile = 'xc.eqx', savepath = None):
    '''
    Constructs the combined :xcquinox.xc.eXC: object from previously-created and separate :xcquinox.net.eX: and :xquinox.net.eC: objects

    :param level: one of ['GGA', 'MGGA', 'NONLOCAL', 'NL'], indicating the desired rung of Jacob's Ladder. NONLOCAL = NL
    :type level: str
    :param x_net_path: Location of the saved exchange network. Must have a {configfile}.pkl parameter file within.
    :type x_net_path: str
    :param c_net_path: Location of the saved correlation network. Must have a {configfile}.pkl parameter file within.
    :type c_net_path: str
    :param configfile: Name for the configuration file, needed when reading in the network to re-generate the same structure, defaults to 'network.config'
    :type configfile: str, optional
    :param xdsfile: Name for the exchange network file, needed when reading in the network overwrite generated random weights, defaults to 'xc.eqx'
    :type xdsfile: str, optional
    :param cdsfile: Name for the correlation network file, needed when reading in the network overwrite generated random weights, defaults to 'xc.eqx'
    :type cdsfile: str, optional
    :param savepath: Location to save the generated network and associated config file, defaults to None
    :type savepath: str, optional
    :return: The resulting exchange-correlation functional.
    :rtype: :xcquinox.xc.eXC:

    '''
    level_dict = {'GGA':2, 'MGGA':3, 'NONLOCAL':4, 'NL':4}
    try:
        with open(os.path.join(x_net_path, configfile+'.pkl'), 'rb') as f:
            xparams = pickle.load(f)
        with open(os.path.join(c_net_path, configfile+'.pkl'), 'rb') as f:
            cparams = pickle.load(f)
    except:
        print('BOTH exchange and correlation networks require a network.config.pkl file to generate the XC functional object.')
        raise
    #create the network to generate the descriptors for saving
    xnet = get_net(xorc='X', level=level, net_path = x_net_path)
    cnet = get_net(xorc='C', level=level, net_path = c_net_path)

    if xdsfile:
        xnet = eqx.tree_deserialise_leaves(os.path.join(x_net_path, xdsfile), xnet)
    if cdsfile:
        cnet = eqx.tree_deserialise_leaves(os.path.join(c_net_path, cdsfile), cnet)

    xc = xce.xc.eXC(grid_models = [xnet, cnet], heg_mult = True, level = level_dict[level.upper()])

    if savepath:
        try:
            os.makedirs(savepath)
        except Exception as e:
            print(e)
            print(f'Exception raised in creating {savepath}.')
        with open(os.path.join(savepath, 'x'+configfile+'.pkl'), 'wb') as f:
            pickle.dump(xparams, f)
        with open(os.path.join(savepath, 'c'+configfile+'.pkl'), 'wb') as f:
            pickle.dump(cparams, f)
        eqx.tree_serialise_leaves(os.path.join(savepath, 'xc.eqx'), xc)
    return xc

def get_xcfunc(level, xc_net_path, configfile = 'network.config', xcdsfile = 'xc.eqx'):
    '''
    Retrieves an XC functional object based on configuration files in given directory.

    :param level: one of ['GGA', 'MGGA', 'NONLOCAL', 'NL'], indicating the desired rung of Jacob's Ladder. NONLOCAL = NL
    :type level: str
    :param xc_net_path: Location of the saved functional. Must have BOTH a 'x'+{configfile}+'.pkl' and 'c'+{configfile}+'.pkl' parameter files within.
    :type xc_net_path: str
    :param configfile: Base name for the configuration files to be read in, defaults to 'network.config'
    :type configfile: str, optional
    :param xcdsfile: If present, the network weights will be overwritten by what's present in this file, defaults to 'xc.eqx'
    :type xcdsfile: str, optional
    :return: The loaded functional
    :rtype: :xcquinox.xc.eXC:
    '''
    level_dict = {'GGA':2, 'MGGA':3, 'NONLOCAL':4, 'NL':4}
    try:
        with open(os.path.join(xc_net_path, 'x'+configfile+'.pkl'), 'rb') as f:
            xparams = pickle.load(f)
        with open(os.path.join(xc_net_path, 'c'+configfile+'.pkl'), 'rb') as f:
            cparams = pickle.load(f)
    except:
        print('Error in opening separate exchange/correlation configuration files. Both must be present to re-create the network architecture.')
        raise

    #create the network to generate the descriptors for saving
    xnet = get_net(xorc='X', level=level, net_path = xc_net_path, configfile='x'+configfile, netfile=None)
    cnet = get_net(xorc='C', level=level, net_path = xc_net_path, configfile='c'+configfile, netfile=None)

    xc = xce.xc.eXC(grid_models = [xnet, cnet], heg_mult = True, level = level_dict[level.upper()])
    if xcdsfile:
        xc = eqx.tree_deserialise_leaves(os.path.join(xc_net_path, xcdsfile), xc)

    return xc
    

# Populate `xcquinox/scripts/script_data/ctests/ran` with X/C Networks

In [25]:
rseeds = [92017, 17920]
dirs = ['gga', 'mgga', 'nl']
constrs = ['c', 'nc']
randir = '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran'
DEPTH = 3
NHIDDEN = 16
for didx, direc in enumerate(dirs):
    for sidx, seed in enumerate(rseeds):
        for cidx, con in enumerate(constrs):
            xsubdir = f'x_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            csubdir = f'c_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            print(xsubdir, csubdir)
            # xfp = os.path.join(randir, xsubdir)
            # cfp = os.path.join(randir, csubdir)
            xfp = None
            cfp = None
            if con == 'c':
                print('Constrained network generation')
                #leave inputs as none so the defaults are imposed, which use constraints   
                x, xc = make_net('x', level=direc, depth=DEPTH, nhidden=NHIDDEN, random_seed = seed, savepath = xfp)
                c, cc = make_net('c', level=direc, depth=DEPTH, nhidden=NHIDDEN, random_seed = seed, savepath = cfp)
            if con == 'nc':
                print('Non-constrained network generation')
                #manually set ueg_limit/spin_scaling/lobs/use/ninputs
                ueg = False
                lob = 0
                ss = False
                use = []
                if direc == 'gga':
                    ninput = 3
                elif direc == 'mgga':
                    ninput = 4
                elif direc == 'nl':
                    ninput = 16
                print('Making exchange network in {}'.format(xsubdir))
                x, xc = make_net('x', level=direc, depth=DEPTH, nhidden=NHIDDEN,
                                 ninput = ninput, use = use, spin_scaling = ss, lob = lob, ueg_limit = ueg,
                                 random_seed = seed, savepath = xfp)
                print('Making correlation network in {}'.format(csubdir))
                c, cc = make_net('c', level=direc, depth=DEPTH, nhidden=NHIDDEN,
                                 ninput = ninput, use = use, spin_scaling = ss, lob = lob, ueg_limit = ueg,
                                 random_seed = seed, savepath = cfp)
                
                

x_3_16_c0_gga c_3_16_c0_gga
Constrained network generation
[Errno 17] File exists: '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/x_3_16_c0_gga'
Exception raised in creating /home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/x_3_16_c0_gga.
[Errno 17] File exists: '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/c_3_16_c0_gga'
Exception raised in creating /home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/c_3_16_c0_gga.
x_3_16_nc0_gga c_3_16_nc0_gga
Non-constrained network generation
Making exchange network in x_3_16_nc0_gga
[Errno 17] File exists: '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/x_3_16_nc0_gga'
Exception raised in creating /home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/x_3_16_nc0_gga.
Making correlation network in c_3_16_nc0_gga
[Errno 17] File exists: '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran/c_3_16

# Populate `xcquinox/scripts/script_data/ctests/ran` with Combined X+C Networks

In [3]:
rseeds = [92017, 17920]
dirs = ['gga', 'mgga', 'nl']
constrs = ['c', 'nc']
randir = '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/ran'
DEPTH = 3
NHIDDEN = 16
for didx, direc in enumerate(dirs):
    for sidx, seed in enumerate(rseeds):
        for cidx, con in enumerate(constrs):
            xsubdir = f'x_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            csubdir = f'c_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            xcsubdir = f'xc_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            print(xsubdir, csubdir)
            # xfp = os.path.join(randir, xsubdir)
            # cfp = os.path.join(randir, csubdir)
            xc = xce.xc.make_xcfunc(level = direc.upper(),
                                    x_net_path = os.path.join(randir, xsubdir),
                                    c_net_path = os.path.join(randir, csubdir),
                                    savepath = os.path.join(randir, xcsubdir))

x_3_16_c0_gga c_3_16_c0_gga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_nc0_gga c_3_16_nc0_gga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_c1_gga c_3_16_c1_gga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_nc1_gga c_3_16_nc1_gga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_c0_mgga c_3_16_c0_mgga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_nc0_mgga c_3_16_nc0_mgga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_c1_mgga c_3_16_c1_mgga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_nc1_mgga c_3_16_nc1_mgga
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
SINGLE NETFILE MATCH FOUND. DESERIALIZING...
x_3_16_c0_nl c_3_16_c0_nl
SINGLE NETFILE

# Populate `xcquinox/scripts/script_data/ctests/pt` with Combined X+C Networks

In [3]:
rseeds = [92017, 17920]
dirs = ['gga', 'mgga', 'nl']
constrs = ['c', 'nc']
randir = '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/pt/pbe0/'
# randir = '/home/awills/Documents/Research/xcquinox/scripts/script_data/ctests/pt/scan/'
DEPTH = 3
NHIDDEN = 16
for didx, direc in enumerate(dirs):
    for sidx, seed in enumerate(rseeds):
        for cidx, con in enumerate(constrs):
            xsubdir = f'x_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            csubdir = f'c_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            xcsubdir = f'xc_{DEPTH}_{NHIDDEN}_{con}{sidx}_{direc}'
            print(xsubdir, csubdir)
            # xfp = os.path.join(randir, xsubdir)
            # cfp = os.path.join(randir, csubdir)
            xc = xce.xc.make_xcfunc(level = direc.upper(),
                                    x_net_path = os.path.join(randir, xsubdir),
                                    c_net_path = os.path.join(randir, csubdir),
                                    savepath = os.path.join(randir, xcsubdir),
                                   xdsfile = '', cdsfile = '')

x_3_16_c0_gga c_3_16_c0_gga
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.1801
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.17
x_3_16_nc0_gga c_3_16_nc0_gga
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.2424
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.24
x_3_16_c1_gga c_3_16_c1_gga
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.1636
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.2498
x_3_16_nc1_gga c_3_16_nc1_gga
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.2412
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.2498
x_3_16_c0_mgga c_3_16_c0_mgga
NETFILE MATCHES FOUND -- MULTIPLE. SELECTING LAST ONE.
ATTEMPTING TO DESERIALIZE xc.eqx.1402
NETFILE MATCHES FOUND